# Lifecycle Protocol

> Protocol for plugins that manage child processes, containers, or other external resources

In [ ]:
#| default_exp protocols.lifecycle

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Protocol, runtime_checkable, List, Dict, Any

from cjm_fasthtml_plugins.core.execution_mode import PluginExecutionMode

## Lifecycle-Aware Plugin Protocol

This protocol defines the interface for plugins that manage external resources like:
- Child processes (e.g., vLLM servers)
- Docker containers
- Conda environments
- Remote/cloud resources

Plugins implementing this protocol provide information about their managed resources,
enabling proper resource tracking and cleanup.

In [ ]:
#| export
@runtime_checkable
class LifecycleAwarePlugin(Protocol):
    """Protocol for plugins that manage external resources.
    
    Plugins implementing this protocol provide information about
    child processes, containers, or other resources they manage.
    
    This enables:
    - Resource tracking across the application
    - Proper cleanup when stopping plugins
    - Conflict detection for GPU/memory usage
    - Cost tracking for cloud resources
    
    Example:
        ```python
        class VoxtralVLLMPlugin(TranscriptionPlugin, LifecycleAwarePlugin):
            def get_execution_mode(self) -> PluginExecutionMode:
                return PluginExecutionMode.SUBPROCESS
            
            def get_child_pids(self) -> List[int]:
                if not self.server or not self.server.process:
                    return []
                return [self.server.process.pid]
            
            def get_managed_resources(self) -> Dict[str, Any]:
                return {
                    'server_url': self.server.base_url,
                    'is_running': self.server.is_running()
                }
            
            def force_cleanup(self) -> None:
                if self.server:
                    self.server.stop()
        ```
    """
    
    def get_execution_mode(self) -> PluginExecutionMode:
        """Get the execution mode of this plugin.
        
        Returns:
            PluginExecutionMode indicating how this plugin executes
        """
        ...
    
    def get_child_pids(self) -> List[int]:
        """Get PIDs of any child processes managed by this plugin.
        
        For plugins that spawn subprocesses (e.g., vLLM servers), this
        should return all child process PIDs for resource tracking.
        
        Returns:
            List of process IDs (empty list if no child processes)
        """
        ...
    
    def get_managed_resources(self) -> Dict[str, Any]:
        """Get information about managed resources.
        
        This can include:
        - Server URLs and ports
        - Container IDs
        - Conda environment names
        - Status information
        - Any other plugin-specific resource info
        
        Returns:
            Dictionary with resource information
        """
        ...
    
    def force_cleanup(self) -> None:
        """Force cleanup of all managed resources.
        
        This should be more aggressive than regular cleanup(),
        killing processes, stopping containers, etc. Used for
        emergency shutdown scenarios.
        """
        ...

## Example Implementation

Here's how a plugin with a subprocess (like vLLM server) would implement this protocol:

In [ ]:
# Example: Simple subprocess-based plugin
import subprocess
from typing import Optional

class MockVLLMPlugin:
    """Mock plugin that spawns a subprocess."""
    
    def __init__(self):
        self.process: Optional[subprocess.Popen] = None
    
    def start_server(self):
        """Start mock server process."""
        # In reality, this would start vLLM server
        self.process = subprocess.Popen(['sleep', '3600'])
        print(f"Started server with PID: {self.process.pid}")
    
    # Implement LifecycleAwarePlugin protocol
    
    def get_execution_mode(self) -> PluginExecutionMode:
        return PluginExecutionMode.SUBPROCESS
    
    def get_child_pids(self) -> List[int]:
        if self.process and self.process.poll() is None:
            return [self.process.pid]
        return []
    
    def get_managed_resources(self) -> Dict[str, Any]:
        return {
            'server_url': 'http://localhost:8000',
            'is_running': self.process is not None and self.process.poll() is None,
            'pid': self.process.pid if self.process else None
        }
    
    def force_cleanup(self) -> None:
        if self.process:
            self.process.kill()
            self.process.wait()
            print("Killed server process")

In [ ]:
# Test the mock plugin
plugin = MockVLLMPlugin()

# Check it implements the protocol
print(f"Implements protocol: {isinstance(plugin, LifecycleAwarePlugin)}")

# Start server
plugin.start_server()

# Get execution mode
print(f"Execution mode: {plugin.get_execution_mode().value}")

# Get child PIDs
pids = plugin.get_child_pids()
print(f"Child PIDs: {pids}")

# Get managed resources
resources = plugin.get_managed_resources()
print(f"Managed resources: {resources}")

# Cleanup
plugin.force_cleanup()
print(f"After cleanup, child PIDs: {plugin.get_child_pids()}")

Implements protocol: True
Started server with PID: 140976
Execution mode: subprocess
Child PIDs: [140976]
Managed resources: {'server_url': 'http://localhost:8000', 'is_running': True, 'pid': 140976}
Killed server process
After cleanup, child PIDs: []


## Helper Functions

Utilities for working with lifecycle-aware plugins.

In [ ]:
#| export
def is_lifecycle_aware(plugin: Any) -> bool:
    """Check if a plugin implements the LifecycleAwarePlugin protocol.
    
    Args:
        plugin: Plugin instance to check
    
    Returns:
        True if plugin implements the protocol
    """
    return isinstance(plugin, LifecycleAwarePlugin)

def get_all_managed_pids(plugin: Any) -> List[int]:
    """Get all PIDs managed by a plugin (including children).
    
    Args:
        plugin: Plugin instance
    
    Returns:
        List of all PIDs (empty if plugin not lifecycle-aware)
    """
    if not is_lifecycle_aware(plugin):
        return []
    
    pids = plugin.get_child_pids()
    
    # Could be extended to recursively get children of children
    # using psutil if needed
    
    return pids

In [ ]:
# Test helper functions
plugin = MockVLLMPlugin()
plugin.start_server()

print(f"Is lifecycle aware: {is_lifecycle_aware(plugin)}")
print(f"All managed PIDs: {get_all_managed_pids(plugin)}")

# Test with non-lifecycle-aware object
regular_obj = {"not": "a plugin"}
print(f"\nDict is lifecycle aware: {is_lifecycle_aware(regular_obj)}")
print(f"Dict managed PIDs: {get_all_managed_pids(regular_obj)}")

# Cleanup
plugin.force_cleanup()

Started server with PID: 140977
Is lifecycle aware: True
All managed PIDs: [140977]

Dict is lifecycle aware: False
Dict managed PIDs: []
Killed server process


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()